In [93]:
import fasttext

"""
The training file should be this format:

__label__pp this is my text
__label__psoe this is also my text
"""

'\nThe training file should be this format:\n\n__label__pp this is my text\n__label__psoe this is also my text\n'

In [195]:
import pandas as pd
import numpy as np
import re
import emoji

#Function to get all emojis in a text.
def split_emojis(text):

    emoji_list = []
    data = re.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

#Function to clean the text of a tweet.
def clean_tweet(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"\n", " ", text)
    #clean_text = re.sub("@.*:", "", text) #This do not improve performance.
    #clean_text = re.sub("@.* ", "", clean_text)
    #clean_text = re.sub("RT", "", clean_text)
    #clean_text = re.sub("\n", " ", clean_text)
    #clean_text = re.sub("//.", "", clean_text)
    #clean_text = re.sub(" +", " ", clean_text)
    #Separate emojis from words.
    """
    sep_emojis_text = ""
    for c in clean_text:
        if c in emoji.UNICODE_EMOJI:
            sep_emojis_text+=' '+c+' '
        else:
            sep_emojis_text+=c
     #This do not improve performance.
    """
    return text

#Read and clean data for each party.

#Podemos.
up_cleaned_data = pd.DataFrame()
up_df = pd.read_csv ('up.csv').head(4950)
up_cleaned_data["tweet"] = up_df["text"].apply(clean_tweet)
up_cleaned_data["label"] = "__label__up"
print(len(up_df))
#Psoe.
psoe_cleaned_data = pd.DataFrame()
psoe_df = pd.read_csv ('psoe.csv').head(4950)
psoe_cleaned_data["tweet"] = psoe_df["text"].apply(clean_tweet)
psoe_cleaned_data["label"] = "__label__psoe"
print(len(psoe_df))
#CS.
cs_cleaned_data = pd.DataFrame()
cs_df = pd.read_csv ('cs.csv').head(4950)
cs_cleaned_data["tweet"] = cs_df["text"].apply(clean_tweet)
cs_cleaned_data["label"] = "__label__cs"
print(len(cs_df))
#PP.
pp_cleaned_data = pd.DataFrame()
pp_df = pd.read_csv ('pp.csv').head(4950)
pp_cleaned_data["tweet"] = pp_df["text"].apply(clean_tweet)
pp_cleaned_data["label"] = "__label__pp"
print(len(pp_df))
#Vox.
vox_cleaned_data = pd.DataFrame()
vox_df = pd.read_csv ('vox.csv').head(4950)
vox_cleaned_data["tweet"] = vox_df["text"].apply(clean_tweet)
vox_cleaned_data["label"] = "__label__vox"
print(len(vox_df))

#Concat all parties data.
data = pd.concat([up_cleaned_data, psoe_cleaned_data, cs_cleaned_data, pp_cleaned_data, vox_cleaned_data])

1
4950
4950
4950
4950


In [202]:
up_df["text"][0]

'RT @JesusCintora: "Sanitarios de la primaria de Madrid llevan a la justicia la gestión de Ayuso. Cerca de 300 profesionales del colectivo A…'

In [203]:
#Split data in training and test sets.
import csv
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data["tweet"], data["label"], test_size=0.15, random_state=42)

training = pd.concat([X_train,y_train], axis=1)
display(training)

test = pd.concat([X_test,y_test], axis=1)
display(test)

#Create training file.
training[training.columns.values[::-1]].to_csv('data.train', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
#data.to_csv('data.train', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

#Create test file.
test[test.columns.values[::-1]].to_csv('data.test', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

,tweet,label
2970,RT @francarrillog: Eligen a los extremos porqu...,__label__cs
275,RT @DipuSG: 🔹¿Eres un alojamiento de la provin...,__label__cs
2264,@FernandoArambur Felicidades por el gran trabajo,__label__pp
244,Es la esperanza de poner coto a un gobierno di...,__label__vox
4323,RT @hugomanchon: La vergüenza hecha Gobierno.,__label__pp
...,...,...
1383,"RT @JorgeBustos1: Veo que a Fernández Vara, qu...",__label__pp
2063,RT @anacamins: Frente a los partidos que justi...,__label__pp
439,RT @CsValladolid: Aprobada la modificación de ...,__label__cs
859,RT @PSOE: Invertir en #TransiciónEcológica y R...,__label__psoe


,tweet,label
4178,RT @jmespejosaav: Es lamentable que Sánchez ha...,__label__cs
2511,@carlosdavidgf Pues hasta que no gobierne el P...,__label__pp
4692,RT @berto_hu: @mikihoyos Ley Wert también fue ...,__label__psoe
3101,RT @pablocast13: ¿Ha dimitido ya Simancas por ...,__label__pp
4402,RT @JMdezOficial: Que va! Como parodia es muy ...,__label__pp
...,...,...
1376,RT @PhilAMellows: La estrategia del Gobierno e...,__label__pp
2506,RT @SaraGimnez: El fracaso escolar condena a m...,__label__cs
2976,"RT @RomeoMo75922153: No lo defiendo, ni mucho ...",__label__vox
45,@felixrubiogama1 @sanchezcastejon @PSOE No le ...,__label__pp


In [204]:
import fasttext
# Load input training set.

#By default, fastText sees each training example only five times during training, which is pretty small. The number of times each examples is seen (also known as the number of epochs), can be increased using the -epoch option
model = fasttext.train_supervised(input="data.train", epoch=30)

In [205]:
#To predict one instance.
print( model.predict("Franco deberia volver a vivir") )
print( model.predict("Los españoles son los mejores 💙") )
print( model.predict("Los españoles son los mejores 💚") )
print( model.predict("España es una gran nacion") )
print( model.predict("Pablo Iglesias es el mejor") )
print( model.predict("Pablo Iglesias es un mierdas") )
print( model.predict("Putos progres de mierda") )
print( model.predict("Viva la guardia civil") )
print( model.predict("Igualdad salarial para las mujeres!") )
print( model.predict("La españa moderada") )
print( model.predict("La mujer a trabajar") )
print( model.predict("Viva la fuerza naranja") )

(('__label__vox',), array([0.99950969]))
(('__label__pp',), array([0.99739093]))
(('__label__vox',), array([0.94602305]))
(('__label__pp',), array([0.65809429]))
(('__label__vox',), array([0.74217999]))
(('__label__vox',), array([0.65079474]))
(('__label__vox',), array([1.00001001]))
(('__label__pp',), array([0.67228228]))
(('__label__pp',), array([0.95372534]))
(('__label__cs',), array([0.99999094]))
(('__label__cs',), array([0.49952245]))
(('__label__pp',), array([0.99870235]))


In [206]:
#To test the test set. (number of samples, precision, recall)
model.test("data.test")

precision = model.test("data.test")[1]
recall = model.test("data.test")[2]

F_score = 2*(precision * recall)/(precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("Fscore:", F_score)

Precision: 0.7353535353535353
Recall: 0.7353535353535353
Fscore: 0.7353535353535352


In [207]:
model.save_model("fastTextModel.model")